## Accquire

**imports**

In [17]:
# data manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# system manipulation
import sys
sys.path.append("./util_")
# import acquire_
# import prepare_

# other
import warnings
warnings.filterwarnings("ignore")

**Get data**

In [9]:
bank = pd.read_csv("./banklist.csv")
bank.head()

,Bank Name†,City†,State†,Cert†,Acquiring Institution†,Closing Date†,Fund
0,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.",1-May-23,10543
1,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.",12-Mar-23,10540
2,Silicon Valley Bank,Santa Clara,CA,24735,FirstñCitizens Bank & Trust Company,10-Mar-23,10539
3,Almena State Bank,Almena,KS,15426,Equity Bank,23-Oct-20,10538
4,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb",16-Oct-20,10537


**Understand data**

In [19]:
# look at the shape of the data
bank.shape

(566, 7)

In [20]:
# what columns are in the data
bank.columns

Index(['Bank Name†', 'City†', 'State†', 'Cert†', 'Acquiring Institution†',
       'Closing Date†', 'Fund'],
      dtype='object')

In [21]:
# count all the null values
bank.isna().sum().values.sum() / len(bank)

0.0

In [22]:
# look at the bak infromation
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Bank Name†              566 non-null    object
 1   City†                   566 non-null    object
 2   State†                  566 non-null    object
 3   Cert†                   566 non-null    int64 
 4   Acquiring Institution†  566 non-null    object
 5   Closing Date†           566 non-null    object
 6   Fund                    566 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 31.1+ KB


In [23]:
# look at the distribution of the data
bank.describe()

,Cert†,Fund
count,566.000000,566.000000
mean,31711.392226,10039.545936
std,16477.485768,1112.975725
min,91.000000,4645.000000
25%,20137.000000,10117.250000
50%,32166.000000,10258.500000
75%,35377.250000,10399.750000
max,59017.000000,10543.000000


In [24]:
# count of numeric columns and object columns
numerics_count = len(bank.select_dtypes("number").columns)
objects_count = len(bank.select_dtypes("object").columns)

print("Numeric col count:", numerics_count)
print("object col count:", objects_count)

Numeric col count: 2
object col count: 5


Take aways:

- I have 566 rows and 7 columns
- 2 numeric columns and 5 string columns
- 0 null count
- I also have descriptive statistics of my data

## Prepare